## Preprocesamiento, transformacion y limpieza

### Venatgium Data Consulting
**Mariana Aguado Muñoz Ledo**

En este notebook se realizará el procesamiento, transformación y limpieza de un conjunto de datos denominados M5 donde se puede ver un conjunto de datos redactando los datos de ventas de diez tiendas walmart y sus ventas a lo largo de 5 años, estos datos fueron extraidos del siguiente [link](https://www.kaggle.com/competitions/m5-forecasting-accuracy/data).

La base de datos cuenta de 5 datasets distintos, solo utilizaremos tres de ellos, que serán: una tabla de calendario (***calendar***), una tabla de la historia de ventas (***sales_train_evaluation***) y una tabla con información de los precios de venta (***sell_prices***). Se cargarán y revisarán los datos, y se hará una transformación y limpieza de llos para finalmente tener un .csv listo para analizarse.

In [2]:
## Librerias 
import pandas as pd
import os
import gc
import matplotlib.pyplot as plt

### Lectura de datos
Comenzaremos leyendo los datasets en format .csv mostrando las primeras filas de los dataset, y observando las columnas y el tipo de columnas presentes.

In [7]:
DATA_PATH = '/Users/marianaaguadomunozledo/Downloads/m5-forecasting-accuracy/'

calendar = pd.read_csv(f'{DATA_PATH}calendar.csv')
sales = pd.read_csv(f'{DATA_PATH}sales_train_evaluation.csv')
sales_train_validation = pd.read_csv(f'{DATA_PATH}sales_train_validation.csv')
sample_submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')
sell_prices = pd.read_csv(f'{DATA_PATH}sell_prices.csv')

In [23]:
calendar.head()
#calendar.info()


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [30]:
sales.head()
#sales_train_evaluation.info()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [19]:
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [32]:
dataframes = {
    "calendar": calendar,
    "sales": sales,
    "sell_prices": sell_prices
}

for df_name, df in dataframes.items():
    print(f"Tipos de datos de {df_name}:\n{df.dtypes}\n")


Tipos de datos de calendar:
date            object
wm_yr_wk         int64
weekday         object
wday             int64
month            int64
year             int64
d               object
event_name_1    object
event_type_1    object
event_name_2    object
event_type_2    object
snap_CA          int64
snap_TX          int64
snap_WI          int64
dtype: object

Tipos de datos de sales:
id          object
item_id     object
dept_id     object
cat_id      object
store_id    object
             ...  
d_1937       int64
d_1938       int64
d_1939       int64
d_1940       int64
d_1941       int64
Length: 1947, dtype: object

Tipos de datos de sell_prices:
store_id       object
item_id        object
wm_yr_wk        int64
sell_price    float64
dtype: object



De los resultados anteriores podemos ver lo siguiente de cada uno de los datasets:

1. **Calendar:** Este dataset contiene información relacionada con fechas y eventos. Incluye variables categóricas como weekday (día de la semana) y event_name_1 (nombres de eventos), así como numéricas como wm_yr_wk (año y semana), month, year, y flags binarios como snap_CA, snap_TX, y snap_WI.

2. **Sales Train Evaluation:** Este dataset detalla las ventas diarias de productos específicos en varias tiendas. Contiene identificadores categóricos como id, item_id (producto), dept_id (departamento), cat_id (categoría), y store_id (tienda). Además, incluye un gran número de columnas numéricas (d_1937 a d_1941) que representan las ventas diarias para diferentes días.

5. **Sell Prices:** Este dataset incluye información sobre los precios de los productos en diferentes tiendas. Contiene identificadores como store_id y item_id, junto con una columna numérica wm_yr_wk que relaciona los precios con una semana específica, y sell_price, que representa el precio de venta como un valor decimal.

### Limpieza y transformación de datos

Despues de una exploración de los datos se puede ver que es posible realizar análisis de distintos tipos dependiendo el propósito y requisitos del estudio. Comenzaremos por prepara los dataframes para despues unirlos en un único df.

Además, se puede ver en los datos que existen muchos campos de columnas que no estan bien clasificados, por ejemplo que se espera que sea numerico y en su lugar es un ***object*** esto puede indicar que hay valores faltantes (o NULL) o algun otro tipo de lectura inusual por lo que se debe tratar antes de analizar los datos. 

Se comenzo por transformar las columnas "d..." del conjunto de ventas para pasar de un formato ancho a un formato largo:

In [8]:
sales_melted = pd.melt(sales, 
                            id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                            var_name='d', 
                            value_name='sales')

Después, se unieron los conjuntos de datos de ventas y calendario:



In [9]:
merged_data = pd.merge(sales_melted, calendar, on='d', how='left')

In [6]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59181090 entries, 0 to 59181089
Data columns (total 21 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   id            object
 1   item_id       object
 2   dept_id       object
 3   cat_id        object
 4   store_id      object
 5   state_id      object
 6   d             object
 7   sales         int64 
 8   date          object
 9   wm_yr_wk      int64 
 10  weekday       object
 11  wday          int64 
 12  month         int64 
 13  year          int64 
 14  event_name_1  object
 15  event_type_1  object
 16  event_name_2  object
 17  event_type_2  object
 18  snap_CA       int64 
 19  snap_TX       int64 
 20  snap_WI       int64 
dtypes: int64(8), object(13)
memory usage: 9.3+ GB


Estamos optimizando el conjunto de datos eliminando columnas redundantes o innecesarias, como identificadores repetitivos (id, d) y datos menos relevantes para el análisis, como nombres de eventos repetitivos (event_name_2, event_type_2) y variables relacionadas con SNAP (snap_CA, snap_TX, snap_WI). Además, convertimos la columna date de tipo object a formato datetime, lo que reduce el uso de memoria y facilita el análisis temporal.

In [10]:
# Convertir 'date' a formato datetime
merged_data['date'] = pd.to_datetime(merged_data['date'])

# Eliminar las columnas seleccionadas
columns_to_drop = [
    'id', 'd', 'weekday', 
    'event_name_2', 'event_type_2', 
    'snap_TX', 'snap_WI'
]
merged_data = merged_data.drop(columns=columns_to_drop)

# Verificar las columnas restantes
print("Columnas restantes en merged_data:")
print(merged_data.columns)


Columnas restantes en merged_data:
Index(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'sales', 'date',
       'wm_yr_wk', 'wday', 'month', 'year', 'event_name_1', 'event_type_1',
       'snap_CA'],
      dtype='object')


Se incorporó el conjunto de precios para tener un solo conjunto de datos final:



In [11]:
final_data = pd.merge(merged_data, sell_prices, how='left', on=['store_id', 'item_id', 'wm_yr_wk'])

In [9]:
final_data.head()

,item_id,dept_id,cat_id,store_id,state_id,sales,date,wm_yr_wk,wday,month,year,event_name_1,event_type_1,snap_CA,sell_price
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,11101,1,1,2011,NaN,NaN,0,NaN
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,11101,1,1,2011,NaN,NaN,0,NaN
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,11101,1,1,2011,NaN,NaN,0,NaN
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,11101,1,1,2011,NaN,NaN,0,NaN
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,11101,1,1,2011,NaN,NaN,0,NaN


Analizamos los valores únicos en algunas columnas de interés para revisar que información podemos obtener de las columnas:

In [10]:
unique_counts = final_data[['dept_id', 'cat_id', 'state_id', 'event_name_1', 'event_type_1']].nunique()
print(unique_counts)

unique_values_dept_id = final_data['dept_id'].unique()
unique_values_cat_id = final_data['cat_id'].unique()
unique_values_state_id = final_data['state_id'].unique()
unique_values_event_name_1 = final_data['event_name_1'].unique()
unique_values_event_type_1 = final_data['event_type_1'].unique()
unique_values_snap_CA = final_data['snap_CA'].unique()


# Imprimir algunos de los resultados si es necesario
print("Valores únicos en dept_id:", unique_values_dept_id[:10])  # Solo los primeros 10
print("Valores únicos en cat_id:", unique_values_cat_id[:10])
print("Valores únicos en state_id:", unique_values_state_id[:10])
print("Valores únicos en event_name_1:", unique_values_event_name_1[:10])
print("Valores únicos en event_type_1:", unique_values_event_type_1[:10])
print("Valores únicos en snap_CA:", unique_values_snap_CA[:10])



dept_id          7
cat_id           3
state_id         3
event_name_1    30
event_type_1     4
dtype: int64
Valores únicos en dept_id: ['HOBBIES_1' 'HOBBIES_2' 'HOUSEHOLD_1' 'HOUSEHOLD_2' 'FOODS_1' 'FOODS_2'
 'FOODS_3']
Valores únicos en cat_id: ['HOBBIES' 'HOUSEHOLD' 'FOODS']
Valores únicos en state_id: ['CA' 'TX' 'WI']
Valores únicos en event_name_1: [nan 'SuperBowl' 'ValentinesDay' 'PresidentsDay' 'LentStart' 'LentWeek2'
 'StPatricksDay' 'Purim End' 'OrthodoxEaster' 'Pesach End']
Valores únicos en event_type_1: [nan 'Sporting' 'Cultural' 'National' 'Religious']
Valores únicos en snap_CA: [0 1]


Para realizar este analisis se tomarán únicamente las tiendas en el estado de California, esto con el supuesto de que se realiza el análisis para un directivo de Walmart en CA.

In [12]:
df_filtered = final_data[final_data['state_id'] == 'CA']

Y se elimna las variables que no necesitamos y se libera memoria para continuar trabajandi

In [13]:
del final_data

In [14]:
gc.collect()

458

Eliminamos otras columnas innecesarias y variables que ya no se necesitan

In [15]:
columns_to_drop = ['store_id', 'state_id', 'wm_yr_wk']
df = df_filtered.drop(columns=columns_to_drop)

del columns_to_drop


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23672436 entries, 0 to 59162795
Data columns (total 12 columns):
 #   Column        Dtype         
---  ------        -----         
 0   item_id       object        
 1   dept_id       object        
 2   cat_id        object        
 3   sales         int64         
 4   date          datetime64[ns]
 5   wday          int64         
 6   month         int64         
 7   year          int64         
 8   event_name_1  object        
 9   event_type_1  object        
 10  snap_CA       int64         
 11  sell_price    float64       
dtypes: datetime64[ns](1), float64(1), int64(5), object(5)
memory usage: 2.3+ GB


## Limpieza de datos

Se empezará por revisar y manejar los datos faltantes

In [3]:
df = pd.read_csv('/Users/marianaaguadomunozledo/Downloads/df_ventagium.csv')


/var/folders/ym/fgv8253x1zzfjqd3mcbp816m0000gn/T/ipykernel_66864/3045100278.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/marianaaguadomunozledo/Downloads/df_ventagium.csv')


In [21]:
missing_data = df.isnull().sum().sort_values(ascending=False)

print("Valores faltantes en cada columna:")
print(missing_data)


Valores faltantes en cada columna:
event_name_1    21745468
event_type_1    21745468
item_id                0
dept_id                0
cat_id                 0
sales                  0
date                   0
wday                   0
month                  0
year                   0
snap_CA                0
sell_price             0
dtype: int64


Como podemos ver, únicamente tenemos valores nulos en los eventos (pues no hay eventos a diario) y en *sell_price*. Se imputará los datos nulos empleando la mediana del precio según el producto en cuestión, garantizando que los valores sean reemplazados con un estimado apropiado.

In [ ]:
df['sell_price'] = df.groupby('item_id')['sell_price'].transform(lambda x: x.fillna(x.median()))
df['sell_price'].isnull().sum()

In [20]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 23672436 entries, 0 to 59162795
Data columns (total 12 columns):
 #   Column        Dtype         
---  ------        -----         
 0   item_id       object        
 1   dept_id       object        
 2   cat_id        object        
 3   sales         int64         
 4   date          datetime64[ns]
 5   wday          int64         
 6   month         int64         
 7   year          int64         
 8   event_name_1  object        
 9   event_type_1  object        
 10  snap_CA       int64         
 11  sell_price    float64       
dtypes: datetime64[ns](1), float64(1), int64(5), object(5)
memory usage: 2.3+ GB


Finalmente, revisemos si existen duplicados:

In [22]:
duplicated_rows = df.duplicated()

# Contar el número de filas duplicadas
num_duplicated_rows = df.duplicated().sum()
print(f"Número de filas duplicadas: {num_duplicated_rows}")

# Mostrar las filas duplicadas si existen
df_duplicated = df[df.duplicated()]
print(df_duplicated)

Número de filas duplicadas: 11088680
                item_id    dept_id   cat_id  sales       date  wday  month  \
3049      HOBBIES_1_001  HOBBIES_1  HOBBIES      0 2011-01-29     1      1   
3050      HOBBIES_1_002  HOBBIES_1  HOBBIES      0 2011-01-29     1      1   
3051      HOBBIES_1_003  HOBBIES_1  HOBBIES      0 2011-01-29     1      1   
3053      HOBBIES_1_005  HOBBIES_1  HOBBIES      0 2011-01-29     1      1   
3054      HOBBIES_1_006  HOBBIES_1  HOBBIES      0 2011-01-29     1      1   
...                 ...        ...      ...    ...        ...   ...    ...   
59162785    FOODS_3_817    FOODS_3    FOODS      0 2016-05-22     2      5   
59162788    FOODS_3_820    FOODS_3    FOODS      1 2016-05-22     2      5   
59162789    FOODS_3_821    FOODS_3    FOODS      0 2016-05-22     2      5   
59162792    FOODS_3_824    FOODS_3    FOODS      0 2016-05-22     2      5   
59162795    FOODS_3_827    FOODS_3    FOODS      0 2016-05-22     2      5   

          year event_name_

Encontramos 11088680 filas duplicadas, tienen que ser eliminadas:


In [23]:
df = df.drop_duplicates()

num_duplicated_rows = df.duplicated().sum()
print(f"Número de filas duplicadas después de eliminar: {num_duplicated_rows}")

Número de filas duplicadas después de eliminar: 0


Exportamos el conjunto de datos final

In [24]:
df.to_csv('/Users/marianaaguadomunozledo/Downloads/df_ventagium.csv', index=False)
